In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
# read master sequestration file from ~/Downloads/master_sequestration_locations_68089_2023-03-15.tsv file
master_sequestration_file = Path.home() / "Downloads" / "master_sequestration_locations_68089_2023-03-15.tsv"
master_sequestration = pd.read_csv(master_sequestration_file, sep="\t")
open_stuff = master_sequestration.loc[master_sequestration["dataset"] == "Open", "case_ids"]

In [ ]:
# read all files from ~/Downloads and create a dataframe for each file in a dictionary
manifests_list = list(Path("~/Downloads").expanduser().glob("**/*.tsv"))
# filter only ones that match this name "new_image_manifest_RSNA"
manifests_list = [x for x in manifests_list if x.name.startswith("new_image_manifest_RSNA")]

In [ ]:
dfs = {f.name: pd.read_csv(f, sep="\t") for f in manifests_list}

In [ ]:
# filter dfs by case_ids from master_sequestration DataFrame
new_dfs = {k: v[v["case_ids"].isin(open_stuff)] for k, v in dfs.items()}

In [ ]:
for k in dfs:
    print(k, dfs[k].shape[0], new_dfs[k].shape[0])

In [ ]:
new_dfs

In [ ]:
new_dfs["new_image_manifest_RSNA_20230131.tsv"]

In [ ]:
for k, v in new_dfs.items():
    tmp = v
    # tmp["storage_urls"].str.replace("storage.ir.rsna.ai", "external-data-midrc-replication")
    tmp["storage_urls"].str.replace("s3://storage.ir.rsna.ai/", "").to_csv(Path.home() / "Downloads" / k, sep="\t", index=False, header=False)